In [70]:
import os
import pandas as pd
from sklearn.datasets import load_iris
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv') 
#엑셀 다운로드

## Feature Engineering

* 상관관계 확인

In [71]:
corr_matrix = train.corr()

In [72]:
train_data = train.drop("Activity", axis=1) # drop labels for training set
train_data = train_data.drop("subject", axis=1)
train_target= train["Activity"].copy()
test_data = test.drop("Activity", axis=1) # drop labels for training set
test_data = test_data.drop("subject", axis=1)
test_target= test["Activity"].copy()

* 새로운 특성 만들기

In [73]:
'''try:
    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20
import numpy as np
cat_encoder = OneHotEncoder(sparse=False)
train_target_cat = cat_encoder.fit_transform(train_target)
test_target_cat = cat_encoder.fit_transform(test_target)'''

'try:\n    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20\n    from sklearn.preprocessing import OneHotEncoder\nexcept ImportError:\n    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20\nimport numpy as np\ncat_encoder = OneHotEncoder(sparse=False)\ntrain_target_cat = cat_encoder.fit_transform(train_target)\ntest_target_cat = cat_encoder.fit_transform(test_target)'

# Feature Scaling

### StandardScaler

In [74]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()
train_data_scale = scaler.fit_transform(train_data.astype(np.float32))
test_data_scale = scaler.fit_transform(test_data.astype(np.float32))

# TrainMode

## DecisionTree

In [75]:
'''from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(max_depth=20, random_state=42)
tree_clf.fit(train_data, train_target)'''

'from sklearn.tree import DecisionTreeClassifier\ntree_clf = DecisionTreeClassifier(max_depth=20, random_state=42)\ntree_clf.fit(train_data, train_target)'

* defult : 0.86
* 20 : 0.86

## RandomForest

In [76]:
'''from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, max_depth=20,n_jobs=-1, random_state=42 )
forest_clf.fit(train_data_scale, train_target)'''

'from sklearn.ensemble import RandomForestClassifier\nforest_clf = RandomForestClassifier(n_estimators=100, max_depth=20,n_jobs=-1, random_state=42 )\nforest_clf.fit(train_data_scale, train_target)'

* 스케일링, one hot : 0.89
* one hot : 0.9
* 없음: 0.92
* 스케일링 : 0.93
* n_estimators=2000 : 

## SVM

**1. SVC**

In [77]:
from sklearn.svm import SVC
poly_kernel_svm_clf = SVC(kernel="poly", degree=2, coef0=1, C=50, gamma='auto', probability= True)
poly_kernel_svm_clf.fit(train_data,train_target) 

SVC(C=50, cache_size=200, class_weight=None, coef0=1,
    decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

* 0.96 

**2.RFD**

In [78]:
'''from sklearn.svm import SVC
rbf_kernel_svm_clf = SVC(degree=3, kernel="rbf", gamma=0.005, C=100)
rbf_kernel_svm_clf.fit(train_data, train_target) '''

'from sklearn.svm import SVC\nrbf_kernel_svm_clf = SVC(degree=3, kernel="rbf", gamma=0.005, C=100)\nrbf_kernel_svm_clf.fit(train_data, train_target) '

* 스케일링 : 0.92
* 없음 ; 0.96

**3. LinearSVC**

In [79]:
'''from sklearn.svm import LinearSVC
svm_clf= LinearSVC(loss="hinge", random_state=42)
svm_clf.fit(train_data, train_target)'''

'from sklearn.svm import LinearSVC\nsvm_clf= LinearSVC(loss="hinge", random_state=42)\nsvm_clf.fit(train_data, train_target)'

## SGD

In [80]:
'''from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=5, tol=-np.infty, random_state=42)
sgd_clf.fit(train_data, train_target)'''

'from sklearn.linear_model import SGDClassifier\n\nsgd_clf = SGDClassifier(max_iter=5, tol=-np.infty, random_state=42)\nsgd_clf.fit(train_data, train_target)'

* 0.94

## Logistic Regression

In [81]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=50 ,random_state=42, max_iter=5000)
log_reg.fit(train_data, train_target)

LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

* 0.96

## AdaBoost

In [82]:
'''from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=50), n_estimators=5000,
    algorithm="SAMME.R", learning_rate=0.1, random_state=42)
ada_clf.fit(train_data, train_target)'''

'from sklearn.ensemble import AdaBoostClassifier\nfrom sklearn.tree import DecisionTreeClassifier\n\nada_clf = AdaBoostClassifier(\n    DecisionTreeClassifier(max_depth=50), n_estimators=5000,\n    algorithm="SAMME.R", learning_rate=0.1, random_state=42)\nada_clf.fit(train_data, train_target)'

##  GradientBoosting

In [ ]:
'''from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(max_depth=5, n_estimators=100, random_state=42)
gbrt.fit(train_data, train_target)'''

## Voting classifiers

In [84]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('svc', poly_kernel_svm_clf) ,('lg', log_reg)], voting= 'soft', n_jobs=-1)
voting_clf.fit(train_data, train_target)

VotingClassifier(estimators=[('svc',
                              SVC(C=50, cache_size=200, class_weight=None,
                                  coef0=1, decision_function_shape='ovr',
                                  degree=2, gamma='auto', kernel='poly',
                                  max_iter=-1, probability=True,
                                  random_state=None, shrinking=True, tol=0.001,
                                  verbose=False)),
                             ('lg',
                              LogisticRegression(C=50, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=5000,
                                                 multi_class='multinomial',
                                                 n_jobs=None, penalty='l2',
                                                 rand

# Performance Measure

## Cross-validation

In [85]:
'''from sklearn.model_selection import cross_val_score
cross_val_score(voting_clf, train_data, train_target, cv=10, scoring="accuracy")'''

'from sklearn.model_selection import cross_val_score\ncross_val_score(voting_clf, train_data, train_target, cv=10, scoring="accuracy")'

## 정확도

In [86]:
from sklearn.metrics import accuracy_score
y_pred = voting_clf.predict(test_data)
accuracy_score(test_target, y_pred)

0.9657278588394977

* 스케일링 된 경우: 0.9643705
* 스케일링되지 않은 경우: 0.9657278588394977